In [1]:
from firedrake import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.auto import tqdm, trange

firedrake:WARNING OMP_NUM_THREADS is not set or is set to a value greater than 1, we suggest setting OMP_NUM_THREADS=1 to improve performance
/Users/mh522/firedrake/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# check 
def check_forward(E,nu,strain_tensor):
    lmbda = E*nu/(1+nu)/(1-2*nu)
    mu = E/2/(1+nu)
    # lmbda*tr(eps(v))*Identity(d) + 2*mu*eps(v), 0.1 is the trace of eps(v)
    s = lmbda*np.trace(strain_tensor)*np.eye(2) + 2*mu*strain_tensor
    return s


In [14]:
## fire version
def get_dataset(ntrain, ntest):
    X, y = [], []

    for i in tqdm(range(ntrain + ntest)):
        # Randomly generate E and nu within given ranges
        E = np.random.uniform(30e3, 90e3)  # Young's modulus in Pa
        nu = np.random.uniform(0.1, 0.3)      # Poisson's ratio

        # Generate diagonal elements
        a11, a12, a22 = [np.random.uniform(-0.1, 0.1) for _ in range(3)]
        # Construct the 2x2 matrix
        strain = np.array([[a11, a12], [a12, a22]])
        
        # strain = project(as_tensor(strain), V_tensor)
        # stress = forward_model(E, nu, strain)
        stress = check_forward(E, nu, strain)
        
        # Flatten and concatenate [E, nu] and strain
        input_data = np.hstack([E, nu, a11, a22, a12])
        
        X.append(input_data)
        y.append([stress[0,0], stress[1,1], stress[0,1]])
        # y.append([stress.dat.data[0,0,0], stress.dat.data[0,1,1], stress.dat.data[0,0,1]])
    # print(y)
    # Convert lists to numpy arrays
    X_train, X_test = np.array(X[:ntrain]), np.array(X[ntrain:])
    y_train, y_test = np.array(y[:ntrain]), np.array(y[ntrain:])
    
    np.save("data/datasets/linear_elasticity/X_train.npy", X_train)
    np.save("data/datasets/linear_elasticity/y_train.npy", y_train)
    np.save("data/datasets/linear_elasticity/X_test.npy", X_test)
    np.save("data/datasets/linear_elasticity/y_test.npy", y_test)

In [15]:
ntrain = 200
ntest = 20
get_dataset(ntrain, ntest)

100%|██████████| 220/220 [00:00<00:00, 24287.93it/s]


# train

In [16]:
# Loading
X_train = np.load("data/datasets/linear_elasticity/X_train.npy", allow_pickle=True)
X_test = np.load("data/datasets/linear_elasticity/X_test.npy", allow_pickle=True)
y_train = np.load("data/datasets/linear_elasticity/y_train.npy", allow_pickle=True)
y_test = np.load("data/datasets/linear_elasticity/y_test.npy", allow_pickle=True)

In [19]:
# Min-Max Normalization
def min_max_normalize(data):
    min_val = np.min(data)
    max_val = np.max(data)
    return (data - min_val) / (max_val - min_val)

X_train_norm = min_max_normalize(X_train)
y_train_norm = min_max_normalize(y_train)
X_test_norm = min_max_normalize(X_test)
y_test_norm = min_max_normalize(y_test)

# Standardization
# Z-score Normalization (Standardization)
def standardize(data):
    mean = np.mean(data)
    std = np.std(data)
    return (data - mean) / std

X_train_norm = standardize(X_train)
y_train_norm = standardize(y_train)
X_test_norm = standardize(X_test)
y_test_norm = standardize(y_test)


In [7]:
import torch.nn as nn
class MLP_model(nn.Module):
    def __init__(self):
        super(MLP_model, self).__init__()
        
        # Define your model architecture
        self.fc1 = nn.Linear(5, 5)  # Input size is 6
        self.fc2 = nn.Linear(5, 3)  # Output size is 4

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def predict(self, x):
        x = self.forward(x)
        return x

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Extract the upper triangle of the matrix
def extract_upper_triangle(matrix):
    indices = np.triu_indices(matrix.shape[1])
    return matrix[:, indices[0], indices[1]]

# Training function
def train_model(model, X_train, y_train, X_test, y_test, epochs=2000, lr=0.00025):
    # Convert data to PyTorch tensors and extract upper triangle
    X_train_tensor = torch.tensor(extract_upper_triangle(X_train), dtype=torch.float32)
    y_train_tensor = torch.tensor(extract_upper_triangle(y_train), dtype=torch.float32)
    X_test_tensor = torch.tensor(extract_upper_triangle(X_test), dtype=torch.float32)
    y_test_tensor = torch.tensor(extract_upper_triangle(y_test), dtype=torch.float32)

    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    losses = []
    test_losses = []
    # Training loop
    for epoch in range(epochs):
        # Forward pass
        model.train()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        losses.append(loss.item())

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            test_outputs = model(X_test_tensor)
            test_loss = criterion(test_outputs, y_test_tensor)
            test_losses.append(test_loss.item())
        # Print loss every 10 epochs
        # if (epoch+1) % 10 == 0:
            # print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
    # Print final loss
    print(f'Final loss: {loss.item():.4f}')
    print(f'Final test loss: {test_loss.item():.4f}')
    plt.plot(test_losses)
    plt.plot(losses)
    # Evaluate the model on the test set

In [20]:
# Initialize the model
model = MLP_model()

# Standardize the data
X_train_norm = standardize(X_train)
y_train_norm = standardize(y_train)
X_test_norm = standardize(X_test)
y_test_norm = standardize(y_test)

# Train the model
train_model(model, X_train_norm, y_train_norm, X_test_norm, y_test_norm)


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [10]:
import numpy as np
import torch.optim as optim
import os

# 初始化
model = MLP_model()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

best_val_loss = float('inf')  # 初始化为正无穷

save_path = "best_model.pth"

epochs = 100
for epoch in range(epochs):
    
    # 训练模型 (使用 train_loader)
    model.train()  # 将模型设为训练模式
    for batch_idx, (inputs, targets) in enumerate(X_train):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    
    # 在验证集上验证模型 (使用 test_loader)
    model.eval()  # 将模型设为评估模式
    val_loss = 0
    with torch.no_grad():  # 在评估模式下，我们不需要计算梯度
        for inputs, targets in X_test:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
    
    val_loss /= len(test_loader)  # 计算平均验证损失

    print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}')

    # 检查是否是最佳模型并保存
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), save_path)
        print(f"Model saved to {save_path}")


NameError: name 'train_loader' is not defined